In [5]:
# Performing all relevant imports
import os
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import glob
from sentence_transformers import SentenceTransformer
import pinecone
import uuid
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.tools import Tool, tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_community.agent_toolkits import create_sql_agent
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, update,select
import yfinance as yf

In [6]:
# Chunking and creating the documents to load
load_dotenv()
loader = DirectoryLoader(r'full_list', glob='./*.txt', loader_cls=TextLoader)
documents = loader.load()
text_splitter =  RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [7]:
texts

[Document(page_content='PART I Item 1 Business Company Background The Company designs manufactures and markets smartphones personal computers tablets wearables and accessories and sells a variety of related services The Companys fiscal year is the 52 or 53week period that ends on the last Saturday of September Products iPhone iPhone is the Companys line of smartphones based on its iOS operating system In October and November 2020 the Company released iPhone 12 iPhone 12 mini iPhone 12 Pro and iPhone 12 Pro Max all with', metadata={'source': 'full_list\\AAPL-10-K-2021-09-25.txt'}),
 Document(page_content='5G technology In September 2021 the Company released iPhone 13 iPhone 13 mini iPhone 13 Pro and iPhone 13 Pro Max Mac Mac is the Companys line of personal computers based on its macOS operating system In November 2020 the Company released new versions of MacBook Air 13inch MacBook Pro and Mac mini and in May 2021 the Company released a redesigned iMac all powered by the Apple M1 chip I

In [8]:
# Creating the vector db
presist_directory = 'db'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=presist_directory
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: 71c4515e****************************************************a103. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# Making a retriever
retriever = vectordb.as_retriever(search_kwargs={'k' : 5})
docs = retriever.get_relevant_documents('What is the net sale for iPhones from Apple in 2022')



c:\Machine Learning\Individual Projects\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# Making a chain
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(),
    chain_type = 'stuff',
    retriever = retriever
)

c:\Machine Learning\Individual Projects\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# Defining all the tools
def database_updater(query : str):
  

   # Initialize the OpenAI model
    llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
    
    # Initialize the database connection
    engine = create_engine('sqlite:///example.db2')
    metadata = MetaData()
    
    # Reflect the database structure
    metadata.reflect(bind=engine)
    
    # Fetch current stock prices from Yahoo Finance
    stock_data = {}
    stock_symbols = ['AAPL', 'MSFT', 'NVDA']
    for symbol in stock_symbols:
        stock = yf.Ticker(symbol)
        stock_data[symbol] = stock.info['currentPrice']
    
    # Assuming there's a table named 'portfolio' with columns 'CLIENT_ID', 'AAPL', 'MSFT', 'NVIDIA', 'STOCKS', 'PRICE'
    portfolio_table = Table('portfolio', metadata, autoload_with=engine)
    
    # Construct SQL query to update stock prices
    with engine.connect() as conn:
        # Fetch the current allocations and stocks from the portfolio table
        columns = [portfolio_table.c.CLIENT_ID, portfolio_table.c.AAPL, portfolio_table.c.MSFT, portfolio_table.c.NVIDIA, portfolio_table.c.STOCKS]
        select_stmt = select(*columns)
        result = conn.execute(select_stmt).fetchall()
        
        for row in result:
            client_id, aapl_allocation, msft_allocation, nvidia_allocation, stocks = row
            price = (aapl_allocation * stock_data['AAPL'] + msft_allocation * stock_data['MSFT'] + nvidia_allocation * stock_data['NVDA']) * stocks
            update_stmt = update(portfolio_table).where(portfolio_table.c.CLIENT_ID == client_id).values(PRICE=price)
            conn.execute(update_stmt)
    
    # Execute the provided query
    db = SQLDatabase.from_uri('sqlite:///example.db2')
    agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
    result = agent_executor.invoke(query)
    
    return result

@tool 
def query_embeddings(query : str):
    """
    This tool is used to perform similarity searches over a vector database
    and return similar responses
    """
    llm_response = qa_chain(query)
    return llm_response['result']

@tool
def search_tool(query : str):
    """Performs online searches to answer the user query. 
    This is only to be used when no relevant results are obtained through a similarity search"""
    search = DuckDuckGoSearchRun()
    answer = query = search.run(query)
    return answer

@tool
def query_database(query : str):
    """
    This function queries the SQL database based on the query that the user inputs.
    This will provide insight into the current client(s) stock portfolio from the portfolio table in the database.

    If the user wants to reallocate stocks, function also allows the user to do so based on the stratergy they use.
    """
    key_words = ['strategy', 'approach', 'method']

    for i in key_words:

        if i not in query:
            llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
            db = SQLDatabase.from_uri('sqlite:///example.db')
            agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
            result = agent_executor.invoke(query)

            return result
        
        else:
            template = """
            You are a helpful assistant who is going to allocate different weights based on context and the invest stratergy that user wants.
            In order to retreieve the context, use the search_tool and the query_embedding tool to search 'Tell me about the fininical siutuation of
            AAPL, MSFT, NVIDIA'. Combining this with the allocation stratergy that the user picks, redistribute the allocation percentages. Give exact figures 
            as to how this is to be used for each stock. You should also give a detailed explanation as to why you came up with these figures. Use the context
            to provide statistics in order to boost your reasoning. Make sure the reasoning is also returned in the final answwer. Use the result to update the curent
            values in the SQL database to those that you outputted.

            Query : {query}
            {answer}
            """
            examples = [
            {
                'query': 'Reallocte the portfolio for client 1 with a balanced strategy',
                'answer': 'The stock allocations for client_id 1 equal across all stocks, totalling 100.'
            },

            {
                'query': 'Reallocte the portfolio for client 1 with a risk-based strategy',
                'answer': """
                                Allocating more to stocks with lower perceived risk. 
                                From my 10k data and online search, Company 1 has strong financials
                                and minimal risk but Comapany 2 has more risk. As a result, the stock
                                allocation fror client_id 1 would have a higher percentage for Company 1
                                and a lower percentage for Company 2. Company 3 would make up the rest of the portfolio.
                                """
            },

            {
                'query': 'Reallocte the portfolio for client 1 with a return-based strategy',
                'answer' : """
                                Allocating more to stocks with higher expected returns.
                                From my 10k data and online search, Company 3 has higher expected growth rates and returns
                                whereas Company 2 has a declining industry position and poor management quality. As a result, the stock
                                allocation fror client_id 1 would have a high percentage for Company 3
                                and a low percentage for Company 2 and Company 1 would make up the rest of the portfolio.
                                """
            }
            ]
            c1 = query_embeddings("what is the current financial outlook of AAPL, MSFT, NVIDIA")
            c2 = search_tool("what is the current financial outlook of AAPL, MSFT, NVIDIA")

            context = c1 + ' ' + c2

            example_prompt = PromptTemplate(
            input_variables=["question", "answer"], template=template
            )

            prompt = FewShotPromptTemplate(
            examples=examples,
            example_prompt=example_prompt,
            suffix="Query: {input}, Context: {context}",
            input_variables=["input", "context"],
        )

            llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)

            chain = prompt | llm

            #print(f"context: {context}")
            ans = chain.invoke({'input' : query, 'context':context})
            
            database_updater(f"set the stock allocations for the client and percentages from {ans.content} provide the sql query to do so and execute it on the database. Also update the Price using these new values, the current number of stocks and the current stock prices.")
            
            return ans.content



tools = [
    Tool(name = 'query_embeddings', func=query_embeddings, description="""Tool for performing similarity searches and returning similar answers to the user's query"""),
    Tool(name='search_tool', func=search_tool, description="""Performs online searches to answer the user query. This is only to be used when no relevant results are obtained through a similarity search"""),
    Tool(name = 'query_database', func=query_database, description="""Performs any action which requires the use of the SQL database including querying it as well as reallocating stocks""")
]

In [ ]:
template = """

You are a helpful assistant how answers any user queries by using a vector databaseto perform similairity searches by using the query_embeddings tool.
Only when this doesn't produce sufficient results, you should use the search_tool tool. Use numbers and figures in all your answers to help answer the user 
query more precisely. ALWAYS USE THE query_embedding tool FIRST.

ONLY use the query_databse if 'client' or 'clients' are mentioned in the query.


Give a detailed and clear answer.
You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer 

Final Answer: the final answer to the ORIGINAL input question

Begin!


Question: {input}

Thought:{agent_scratchpad}
"""

#query = 'What was Apples gross margin in 2022'
#query = 'What is the weather in London today'
query = "Reallocate client 4's stocks using a risk-based approach"
#query = 'Give me all the stock allocations for client 3'
prompt = PromptTemplate(input_variables=['input', 'agent_scratchpad', 'tools', 'tool_names'], template=template)

llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY  '), model='gpt-3.5-turbo', temperature=0.2)

agent = create_react_agent(llm=llm, prompt=prompt, tools=tools)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I see that the query mentions 'client', so I should use the query_database tool for this specific query.

Action: query_database

Action Input: client 4, risk-based approach


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


portfolio
Invoking: `sql_db_schema` with `{'table_names': 'portfolio'}`



CREATE TABLE portfolio (
	"CLiENT_ID" INTEGER, 
	"AAPL" REAL NOT NULL, 
	"MSFT" REAL NOT NULL, 
	"NVIDIA" REAL NOT NULL, 
	PRIMARY KEY ("CLiENT_ID")
)

/*
3 rows from portfolio table:
CLiENT_ID	AAPL	MSFT	NVIDIA
1	0.3	0.5	0.2
2	0.4	0.4	0.2
3	0.5	0.3	0.2
*/
Invoking: `sql_db_query` with `{'query': 'SELECT CLiENT_ID, AAPL, MSFT, NVIDIA FROM portfolio WHERE CLiENT_ID = 4'}`
responded: The `portfolio` table contains information about the investment portfolio of clients. It has columns for `CLiENT_ID`, `AAPL`, `MSFT`, and `NVIDIA`.

I will query the `CLiENT_ID` and the risk-based approach for client 4 from the `portfolio

In [ ]:
query = 'What is the weather today in London'
query_embeddings(query)

"I don't know."

In [ ]:
vectordb.delete_collection

<bound method Chroma.delete_collection of <langchain_community.vectorstores.chroma.Chroma object at 0x000001D144653430>>